# Notebook

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import neat
import matplotlib.pyplot as plt

import pickle
import multimodal_mazes
from tqdm import tqdm

path = '../Results/test6/'

## Initialisations

In [ ]:
# WIP: Initialisations 
path = '../'
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        path + 'neat_config.ini')

p = neat.Population(config)

In [ ]:
# WIP: Initialisations
genome = p.population[6]
node_names = {-1: 'Ch0 L', -2: 'Ch1 L', -3 : 'Ch0 R', -4 : 'Ch1 R', 
              -5: 'Ch0 U', -6: 'Ch1 U', -7 : 'Ch0 D', -8 : 'Ch1 D',
              0 : 'Act L', 1 : 'Act R', 2 : 'Act U', 3 : 'Act D', 4 : 'Wait'}
multimodal_mazes.plot_architecture(genome, config, node_names=node_names)

## Cue fills

In [ ]:
# WIP: path fidelities
noises = np.linspace(start=0.0, stop=0.1, num=18)
channels = [[0,0], [1,0], [0, 1], [1,1]]
results = np.zeros((len(noises), len(channels)))
cols = ['k', "xkcd:dark seafoam", "xkcd:purple", "xkcd:teal blue"]
labels = ['- / -', '+/ -', '- /+', '+/+']

for a, noise in enumerate(noises):

    # Generate mazes
    maze = multimodal_mazes.GeneralMaze(size=11, n_channels=2)
    maze.generate(number=100, noise_scale=0.0)

    # Test agents
    for b, channel in enumerate(channels): 
        agnt = multimodal_mazes.AgentRuleBased(location=None, channels=channel, memory_on=False)
        fitness = multimodal_mazes.eval_fitness(genome=None, config=None, channels=channel, sensor_noise_scale=noise, drop_connect_p=0.0, maze=maze, n_steps=40, agnt=agnt)

        results[a, b] = fitness

for b, channel in enumerate(channels): 
    plt.plot(noises, results[:,b], c=cols[b], label=labels[b])

plt.ylim([0, 1.05])
plt.ylabel('Fitness')
plt.xlabel('Sensor Noise')
plt.legend()

In [ ]:
# WIP: Example mazes 
maze = multimodal_mazes.GeneralMaze(size=11, n_channels=2)
maze.generate(number=1000, noise_scale=0.0)
print(max(maze.fastest_solutions))

In [ ]:
# WIP: Example paths 
n = 0
# path = multimodal_mazes.GeneralMaze.shortest_path(mz=maze.mazes[n], d_map=maze.d_maps[n], start=maze.start_locations[n], exit=maze.goal_locations[n])
agnt = multimodal_mazes.AgentRuleBased(location=None, channels=[1,1], memory_on=False)
time, path = multimodal_mazes.maze_trial(mz=maze.mazes[n], mz_start_loc=maze.start_locations[n], mz_goal_loc=maze.goal_locations[n], channels=[1,1], sensor_noise_scale=0.0, drop_connect_p=0.0, n_steps=150, agnt=agnt)
multimodal_mazes.plot_path(path=path, mz=maze.mazes[n], mz_goal_loc=maze.goal_locations[n], n_steps=len(path))

## Multiple experiments

In [ ]:
# WIP: Comparing 2 experiments with n repeats each 

# Building the feature matricies
import os
paths = ['../Results/test3/', '../Results/test4/']
fitness_cutoff = 1.0

metrics_x, metrics_y, metrics_z = [], [], []
for a, path in enumerate(tqdm(paths)): 

    # Load config data 
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        path + 'neat_config.ini')
    exp_config = multimodal_mazes.load_exp_config(path + 'exp_config.ini')

    # Generate mazes
    maze = multimodal_mazes.TrackMaze(size=exp_config['maze_size'], n_channels=len(exp_config['channels']))
    maze.generate(number=exp_config['n_mazes'], noise_scale=exp_config['maze_noise_scale'], gaps=exp_config['maze_gaps']) 

    # Load data 
    for f in os.listdir(path):
        if f.endswith(".npy"):
            
            print("Testing: " + str(a) + "-" + os.path.splitext(f)[0])
            exp_data = np.load(path + f)
            with open(path + os.path.splitext(f)[0] + '.pickle', 'rb') as file:
                genomes = pickle.load(file)

            # Select top agents 
            top_agents = multimodal_mazes.id_top_agents(fitness_cutoff=fitness_cutoff, exp_data=exp_data, maze=maze, exp_config=exp_config, genomes=genomes, config=config)

            if top_agents:
                # Calculate architecture metrics 
                top_agents_metrics_n, top_agents_metrics_p, mn_keys, mp_keys = multimodal_mazes.architecture_metrics_matrices(agents=top_agents, genomes=genomes, config=config)     
                results_x = np.concatenate((top_agents_metrics_n, top_agents_metrics_p), axis=1) 

                # Store data 
                metrics_x.append(results_x)
                metrics_y.append(np.ones(len(results_x)) * a)
                metrics_z.append(np.ones(len(results_x)) * int(os.path.splitext(f)[0]))

metrics_x = np.concatenate(metrics_x, axis=0) # agents x metrics 
metrics_y = np.concatenate(metrics_y, axis=0) # agents, 
metrics_z = np.concatenate(metrics_z, axis=0) # agents, 
metrics_labels = list(mn_keys) + list(mp_keys) # metrics, 
metrics_type = np.concatenate((np.zeros(len(mn_keys)), np.ones(len(mp_keys)))) # metrics, 

assert len(metrics_x) == len(metrics_y) == len(metrics_z), "Mismatched data?"
assert metrics_x.shape[1] == len(metrics_labels) == len(metrics_type), "Mismatched labels?"

print(np.unique(metrics_y, return_counts=True))

In [ ]:
# WIP: Comparing features 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

model = DecisionTreeClassifier(class_weight="balanced")
scores = cross_val_score(estimator=model, X=metrics_x, y=metrics_y, cv=10)
print(scores)
model.fit(metrics_x, metrics_y)
plt.plot(model.feature_importances_) # sums to 1

In [ ]:
# WIP: Ranking features 
acc_per_metric = []
for f, _ in enumerate(metrics_labels): 
    model = DecisionTreeClassifier(class_weight="balanced", max_depth=1)
    scores = cross_val_score(estimator=model, X=metrics_x[:,f].reshape(-1,1), y=metrics_y, cv=10)
    acc_per_metric.append(scores.mean())

# WIP: Plot ranked features 
f, ax = plt.subplots(1, figsize=(10, 5))
metrics_sorted = np.argsort(acc_per_metric)[::-1]
plt.plot(np.array(acc_per_metric)[metrics_sorted], 'k')
v, c = np.unique(metrics_y, return_counts=True)
plt.hlines(y = max(c) / sum(c), xmin=0, xmax=len(metrics_sorted)-1, color="xkcd:gray", ls="dotted")
plt.xticks(range(len(metrics_labels)), [metrics_labels[l] for l in metrics_sorted], rotation='vertical')
plt.ylabel('Accuracy')

In [ ]:
# WIP: Plot just the best feature 
cols = ['xkcd:grey', '#0ABAB5']

best_feature = metrics_sorted[1]
for b in [0,1]:
    parts = plt.violinplot(dataset=metrics_x[metrics_y==b,:][:, best_feature], positions=[b], showextrema=False, showmedians=True);
    for pc in parts['bodies']:
        pc.set_facecolor(cols[b])
        pc.set_edgecolor(cols[b])
        pc.set_alpha(0.5)

    vp = parts['cmedians']
    vp.set_edgecolor(cols[b])
    vp.set_alpha(1)

plt.ylabel(metrics_labels[best_feature])
plt.xticks([0,1], ['A', 'B'])

In [ ]:
# WIP: All features plot  
cols = ['xkcd:grey', '#0ABAB5']
y_labels = ['Number', 'Value']
offsets = [-0.25, 0.25]

lines = []
f, ax = plt.subplots(1, 2, gridspec_kw={'width_ratios': [0.4, 0.6]}, figsize=(15, 5))
for a in [0,1]: # for each axis 
    plt.sca(ax[a])
    ms = metrics_sorted[metrics_type[metrics_sorted] == a]
    for b in [0,1]: # for each group 
        parts = plt.violinplot(dataset=metrics_x[metrics_y==b,:][:, ms], positions=np.linspace(start=0, stop=len(ms) - 1, num=len(ms))+offsets[b], showextrema=False, showmedians=True);
        for pc in parts['bodies']:
            pc.set_facecolor(cols[b])
            pc.set_edgecolor(cols[b])
            pc.set_alpha(0.5)

        vp = parts['cmedians']
        vp.set_edgecolor(cols[b])
        vp.set_alpha(1)

        if a == 0: lines.append(vp)

    ax[a].set_xticks(np.arange(len(ms)), [metrics_labels[i] for i in ms], rotation='vertical')
    ax[a].set_ylabel(y_labels[a])

# Legend 
plt.legend(lines, ['A, n=' + str(sum(metrics_y == 0)),
                   'B, n=' + str(sum(metrics_y == 1))])

## Single experiments

In [ ]:
# Load config data 
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        path + 'neat_config.ini')

exp_config = multimodal_mazes.load_exp_config(path + 'exp_config.ini')
print('Maze noise: ' + str(exp_config['maze_noise_scale']))
print('Sensor noise: ' + str(exp_config['sensor_noise_scale']))
print('Drop connect p: ' + str(exp_config['drop_connect_p']))

In [ ]:
# Load data 
x = np.load(path + '99.npy')

top_agent = np.where(x['fitness'] == x['fitness'].max())

with open(path + '99.pickle', 'rb') as file:
    genomes = pickle.load(file)

genome_id, genome, channels = genomes[top_agent[0][0]]

print(x[top_agent[0][0]])
print(genome.size())

In [ ]:
# Plotting

# Fitness 
multimodal_mazes.plot_fitness_over_generations(x, plot_species=True)

# Architecture
node_names = {-1: 'Ch0 L', -2: 'Ch1 L', -3 : 'Ch0 R', -4 : 'Ch1 R', 
              -5: 'Ch0 U', -6: 'Ch1 U', -7 : 'Ch0 D', -8 : 'Ch1 D',
              0 : 'Act L', 1 : 'Act R', 2 : 'Act U', 3 : 'Act D', 4 : 'Wait'}
genome = multimodal_mazes.prune_architecture(genome, config)
plt.figure()
multimodal_mazes.plot_architecture(genome, config, node_names=node_names)

In [ ]:
# Top genomes 
fitness_cutoff = 0.9

# Generate mazes
maze = multimodal_mazes.GeneralMaze(size=exp_config['maze_size'], n_channels=len(exp_config['channels']))
maze.generate(number=exp_config['n_mazes'], noise_scale=exp_config['maze_noise_scale'])

# Test agents 
top_agents = multimodal_mazes.id_top_agents(fitness_cutoff=fitness_cutoff, exp_data=x, maze=maze, exp_config=exp_config, genomes=genomes, config=config)
print(len(top_agents))

In [ ]:
# Visualising a path
n=0
time, path = multimodal_mazes.maze_trial(mz=maze.mazes[n], mz_start_loc=maze.start_locations[n], mz_goal_loc=maze.goal_locations[n], channels=[1,1], sensor_noise_scale=0.0, drop_connect_p=0.0, n_steps=30, genome=genome, config=config)
multimodal_mazes.plot_path(path=path, mz=maze.mazes[n], mz_goal_loc=maze.goal_locations[n], n_steps=len(path))

In [ ]:
# Architecture metrics 
top_agents_metrics_n, top_agents_metrics_p, mn_keys, mp_keys = multimodal_mazes.architecture_metrics_matrices(agents=top_agents, genomes=genomes, config=config)

# Plotting 
f, (a0, a1) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [0.4, 0.6]}, figsize=(15, 5))
a0.plot(top_agents_metrics_n.T, c='k', linewidth=1.5, alpha=0.25);
a0.set_xticks(np.arange(len(mn_keys)), mn_keys, rotation='vertical')
a0.set_ylabel('Number')

a1.plot(top_agents_metrics_p.T, c='k', linewidth=1.5, alpha=0.25);
a1.set_xticks(np.arange(len(mp_keys)), mp_keys, rotation='vertical')
a1.set_ylabel('Value')
a1.set_ylim([-0.025, 1.025])

In [ ]:
# WIP: Visualising networks with specific features 
# idx = np.where(top_agents_metrics_n[:,4] == top_agents_metrics_n[:,4].min()) # fewest edges
# idx = np.where(top_agents_metrics_p[:,2] == top_agents_metrics_p[:,2].max()) # most ii connections
idx = np.where(top_agents_metrics_p[:,-2] == top_agents_metrics_p[:,-2].min()) # lowest e:i ratio

# Mode transitivity
# v,c = np.unique(top_agents_metrics_p[:,-3], return_counts=True)
# idx = np.where(top_agents_metrics_p[:,-3] == v[np.argmax(c)]) 

_, genome, _ = genomes[top_agents[idx[0][0]]]
genome = multimodal_mazes.prune_architecture(genome, config)
plt.figure()
multimodal_mazes.plot_architecture(genome, config, node_names=node_names)

## Robustness

In [ ]:
# Robustness to sensor noise
noise_scales = np.linspace(start=0.0, stop=0.5, num=10)
noise_results, noise_baseline = multimodal_mazes.robustness_to_sensor_noise(agents=top_agents, noise_scales=noise_scales, n_mazes=100, exp_config=exp_config, genomes=genomes, config=config)
multimodal_mazes.plot_robustness(condition_values=noise_scales, condition_results=noise_results, condition_label='Sensor noise', noise_baseline=noise_baseline, agents=top_agents, genomes=genomes, config=config, node_names=node_names)
print(min(np.trapz(y=noise_results, x=noise_scales)), max(np.trapz(y=noise_results, x=noise_scales)))

In [ ]:
# Robustness to dropconnect 
drop_scales = np.linspace(start=0.0, stop=1.0, num=10)
drop_results, drop_baseline = multimodal_mazes.robustness_to_drop_connect(agents=top_agents, drop_connect_ps=drop_scales, n_mazes=100, exp_config=exp_config, genomes=genomes, config=config)
multimodal_mazes.plot_robustness(condition_values=drop_scales, condition_results=drop_results, condition_label='Drop connect p', noise_baseline=drop_baseline, agents=top_agents, genomes=genomes, config=config, node_names=node_names)
print(min(np.trapz(y=drop_results, x=drop_scales)), max(np.trapz(y=drop_results, x=drop_scales)))

## Exact baselines

In [ ]:
# # WIP: exact baseline 
# import itertools

# # Parameters 
# n_actions = [-1, 1, 0]
# n_time_steps = 10

# # Enumerate 
# n_action_sequences = len(n_actions) ** n_time_steps 
# n_paths = len(np.unique(n_actions)) ** n_time_steps 

# all_paths = list(itertools.product(np.unique(n_actions), repeat=n_time_steps))
# all_paths = np.array(all_paths, dtype='b')
# all_paths_cu = np.cumsum(all_paths, axis=1) 

# # Boundary conditions 
# all_paths_cu = np.clip(all_paths_cu, a_min=-4, a_max=4)

# fitness = []
# for path in all_paths_cu:
#     if min(path) == -4: # if you've reached the goal 
#         fitness.append(((1 - ((np.where(path == -4)[0][0] - 3) / (n_time_steps - 1 - 3))) + 1.0) * 0.5) # time + path 
#     else:
#         fitness.append((8 - (path[-1] - (-4))) / 8)

# print(np.mean(fitness))
# # exact_baseline = mean(fitness of each path * how often it occurs in the action sequences) 

# # Does this deal with left and right trials correctly? 